In [1]:
import networkx as nx
from types_definition import * 
import vector
from vector import Vector
from rotorconfig import RotorConfig
from rotorgraph import RotorGraph, display_path, all_config_from_recurrent, display_grid
from particleconfig import ParticleConfig
from matrices import Matrix
from arcsum import ArcSum
import numpy as np
from smithnormalform import matrix, snfproblem, z
from otherfunctions import *
import random

import matplotlib.pyplot as plt
from itertools import product


In [2]:
sum1= ArcSum({(1, 0, 0): 1, (2, 0, 0): 1, (3, 0, 0): -1, (3, 1, 0): 1, (3, 2, 0): 1})

Cycle = find_circulation(sum1)


if find_circulation(sum1):
    print(Cycle)


[{(1, 0, 0): 1, (3, 1, 0): 1}, {(2, 0, 0): 1, (3, 2, 0): 1}]


In [3]:
def send_to_sink(n: int,rotor_graph:RotorGraph, arc_sum:ArcSum,node= Node, sink=Node)-> ArcSum:
    """
    In the complete graph
    Gives the sums of arcs obtained by routing a particle on the given Node to the given Sink
    """
    if (1<= node <= n-2) and (sink==0 or sink == n-1):
        edge= (node,sink,0)
    else:
        raise ValueError("node or sink not valid")
    result = arc_sum + ArcSum({rotor_graph.turn(edge):1}) - ArcSum({edge:1})
    
    return result
                


In [9]:

#Test
n=6
sinks = {0,n-1}
list_sinks= list(sinks)
sinks = {0}


rg= RotorGraph.complete_graph_with_sink(n, sinks)
rotor_graph = rg.remove_sink_out_edges(sinks)

#print(sum1)

#print(send_to_sink(n,rotor_graph, sum1,2, 0))
#print(send_to_sink(n,rotor_graph, sum1,2, 3))
#print(list(sinks))


In [12]:
def arcsum_to_rotorconfig(rotor_graph:RotorGraph, arc_sum:ArcSum)->ArcSum:
    compteur = 0

    
    while find_circulation(arc_sum):
        
        compteur +=1
        find=0
        list_circulation = find_circulation(arc_sum)
        for circulation in list_circulation:
            
            potential_result = arc_sum + rotor_graph.linear_turn_vector(circulation) - circulation
            list_coef= list(potential_result.values())
            
            
            #Test if it is a rotor config
            if all(x >= 0 for x in list_coef):
                arc_sum = potential_result
                find=1
                break
            #Test if there is a circulation
            elif find_circulation(potential_result):
                arc_sum = potential_result
                find=1
                
        if find == 0:
            arc_sum = potential_result
        #print(cas)
   
    
    #print("Number of circulation push:", compteur)
    return arc_sum, compteur




In [10]:
# #Test
sum1= ArcSum({(1, 2, 0): 1, (2, 1, 0): 1, (2, 0, 0): -1, (3, 1, 0): 1, (2, 3, 0): 1})

sum2= ArcSum({(1, 2, 0): 0, (2, 1, 0): 1, (2, 3, 0): 0, (3, 1, 0): 0, (2, 4, 0): 1, (2, 0, 0): -1, (1, 3, 0): 1, (3, 2, 0): 1})


Cycle1= find_circulation(sum1)
Cycle2= find_circulation(sum2)

if find_circulation(sum1):
    print("1",Cycle1)

if find_circulation(sum2):
    print("2",Cycle2)

sum=sum2
print(arcsum_to_rotorconfig(rotor_graph, sum))

{(1, 2, 0): 1, (2, 1, 0): 1, (2, 0, 0): -1, (3, 1, 0): 1, (2, 3, 0): 1}
{(1, 2, 0): 1, (2, 1, 0): 1, (2, 0, 0): -1, (3, 1, 0): 1, (2, 3, 0): 1}
3
{(1, 2, 0): 0, (2, 1, 0): 1, (2, 3, 0): 0, (3, 1, 0): 0, (2, 4, 0): 1, (2, 0, 0): -1, (1, 3, 0): 1, (3, 2, 0): 1}
3


1 [{(1, 2, 0): 1, (3, 1, 0): 1, (2, 3, 0): 1}, {(1, 2, 0): 1, (2, 1, 0): 1}]
2 [{(2, 1, 0): 1, (1, 3, 0): 1, (3, 2, 0): 1}]
None
None
({(3, 4, 0): 1, (1, 2, 0): 0, (2, 1, 0): 0, (2, 3, 0): 1, (3, 1, 0): 0, (1, 4, 0): 1, (2, 4, 0): 1, (2, 0, 0): -1, (1, 3, 0): 0, (3, 2, 0): 0}, 1)


3

In [15]:
####Simple ARRIVAL

#Générer une config de rotor au hazard
rho = RotorConfig(rotor_graph)
for i in range(1,n-1):
    eh= random.randint(0, len(rotor_graph.rotor_order[i])-1)
    rho[i]= rotor_graph.rotor_order[i][eh]



#Transformer la config de rotor en somme d'arc
arc_sum = ArcSum(rho)

#choisir un sommet au hazard
node = random.randint(1, n-1)



print("rotor congig: ", rho, " Node ", node)

for sink in list(sinks):
    config_sink = send_to_sink(n,rotor_graph, arc_sum,node, sink)
    print(config_sink)
    potential_config, compteur = arcsum_to_rotorconfig(rotor_graph, config_sink)
    
    list_coef= list(potential_config.values())

    potential_config = ArcSum(potential_config)
    
    if all(x >= 0 for x in list_coef):
        rc= potential_config.is_rotorconfig_of(rotor_graph)
        #print(rc )
        #print("sink", sink)
        f_sink=sink


#config de particule au hazard
sigma = ParticleConfig(rotor_graph)
sigma[node] =1 

###Routage normale avec une config de rotor et de particule aux hasards
#routage aux puits

sigma_f, rho_f, info = rotor_graph.legal_routing(sigma, rho)

#print(rho_f)

#print("Sink 0:", sigma_f[0], " Sink ",n-1, ":", sigma_f[n-1])

print(sigma_f[f_sink]==1, f_sink)

rotor congig:  {1: (1, 5, 0), 2: (2, 1, 0), 3: (3, 5, 0), 4: (4, 0, 0), 5: (5, 0, 0)}  Node  2
{(5, 0, 0): 1, (3, 5, 0): 1, (2, 1, 0): 2, (2, 0, 0): -1, (1, 5, 0): 1, (4, 0, 0): 1}


KeyboardInterrupt: 

In [16]:
list_compteur = []

#print(len(list(rotor_graph.enum_configurations())))
#i=0
for rho in list(rotor_graph.enum_configurations()):
    #i+=1
    #print(i)
    
    #Transformer la config de rotor en somme d'arc
    arc_sum = ArcSum(rho)
    
    #choisir un sommet 
    #node = random.randint(1, n-2)
    for node in range(1,n-1):
        #print("rotor congig: ", rho, " Node ", node)
        f_sink=10
        for sink in list(sinks):
            config_sink = send_to_sink(n,rotor_graph, arc_sum,node, sink)
            potential_config, compteur = arcsum_to_rotorconfig(rotor_graph, config_sink)
            list_compteur.append(compteur)
            list_coef= list(potential_config.values())

            potential_config = ArcSum(potential_config)
            
            if all(x >= 0 for x in list_coef):
                rc= potential_config.is_rotorconfig_of(rotor_graph)
                f_sink=sink


        #config de particule au hazard
        sigma = ParticleConfig(rotor_graph)
        sigma[node] =1 

        ###Routage normale avec une config de rotor et de particule aux hasards
        #routage aux puits

        sigma_f, rho_f, info = rotor_graph.legal_routing(sigma, rho)

        #print(sigma_f[f_sink]==1, f_sink)

        if sigma_f[f_sink]!=1:
            print("pas bon")
            print(rho, node, f_sink)
            print(sigma_f)
            
print(max(list_compteur))
            
    

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [ ]:
####Verification
{1: (1, 0, 0), 2: (2, 1, 0), 3: (3, 0, 0), 4: (4, 3, 0)} 
4
#Générer une config de rotor au hazard
rho = RotorConfig(rotor_graph)
#print(rho)
rho = RotorConfig({1: (1, 0, 0), 2: (2, 1, 0), 3: (3, 0, 0), 4: (4, 3, 0)}  )

arc_sum = ArcSum(rho)

#choisir un sommet au hazard
node = 4


print("rotor config: ", rho, " Node ", node)

f_sink=10
for sink in list(sinks):
    config_sink = send_to_sink(n,rotor_graph, arc_sum,node, sink)
    print(config_sink, sink)
    potential_config, compteur = arcsum_to_rotorconfig(rotor_graph, config_sink)
    #print(potential_config)
    list_coef= list(potential_config.values())

    potential_config = ArcSum(potential_config)
    if all(x >= 0 for x in list_coef):
        rc= potential_config.is_rotorconfig_of(rotor_graph)
        print(rc )
        print("sink", sink)
        f_sink=sink


#config de particule au hazard
sigma = ParticleConfig(rotor_graph)
sigma[node] =1 

###Routage normale avec une config de rotor et de particule aux hasards
#routage aux puits

sigma_f, rho_f, info = rotor_graph.legal_routing(sigma, rho)

#print(rho_f)


#print("Sink 0:", sigma_f[0], " Sink ",n-1, ":", sigma_f[n-1])

print(sigma_f[f_sink]==1, f_sink)

rotor config:  {1: (1, 0, 0), 2: (2, 1, 0), 3: (3, 0, 0), 4: (4, 3, 0)}  Node  4
{(1, 0, 0): 1, (4, 1, 0): 1, (3, 0, 0): 1, (4, 3, 0): 1, (2, 1, 0): 1, (4, 0, 0): -1} 0
{(1, 0, 0): 1, (3, 0, 0): 1, (4, 3, 0): 1, (2, 1, 0): 1, (4, 5, 0): -1, (4, 0, 0): 1} 5
False 10


In [ ]:
sum2=ArcSum({(1, 0, 0): 1, (4, 1, 0): 1, (3, 0, 0): 1, (4, 3, 0): 1, (2, 1, 0): 1, (4, 0, 0): -1})
print(find_circulation(sum2))

# print(arcsum_to_rotorconfig(rotor_graph, sum2))

# sum3=ArcSum({(1, 0, 0): 1, (3, 0, 0): 1, (4, 3, 0): 1, (2, 1, 0): 1, (4, 5, 0): -1, (4, 0, 0): 1})

# print(find_circulation(sum3))

#
#sum3=ArcSum( {(4, 3, 0): 1, (1, 2, 0): 1, (3, 0, 0): 0, (3, 1, 0): -1, (1, 0, 0): 0, (2, 0, 0): 1, (3, 2, 0): 2})
#print(find_circulation(sum3))


[{(1, 0, 0): 1, (4, 1, 0): 1, (4, 0, 0): -1}, {(3, 0, 0): 1, (4, 3, 0): 1, (4, 0, 0): -1}]
